In [ ]:
from principal_DBN_alpha import *
from principal_RBM_alpha import *
from utils import *
from tqdm import tqdm, notebook
import matplotlib.pyplot as plt

# Etude sur Binary AlphaDigit

In [ ]:
data = lire_alpha_digit(caracs, path_data=path_data)

## RBM 

On entraine un RBM sur les données de Binary AlphaDigit puis on genere des images à partir de ce qu'il a appris.

In [ ]:
neurons = 200
epochs = 200
nb_pixels = 20*16
learning_rate = 0.1
gibbs_steps = 200

In [ ]:
data.shape

On entraine un RBM pour chaque caractère.

In [ ]:
list_rbm_caracs = []
for carac in notebook.tqdm(list(caracs.keys()), desc="Characters", unit="char", leave=False):
    data = lire_alpha_digit([carac], path_data)
    nb_pixels = data.shape[1]
    rbm = RBM(p = nb_pixels, q = neurons)
    rbm.train_RBM(x=data, epochs=epochs, lr=learning_rate, show_progress=True)
    list_rbm_caracs.append(rbm)

On genere des images des caractères en utilisant la méthode de Gibbs Sampling.

In [ ]:
fig, axs = plt.subplots(4, 9, figsize=(10, 7), sharex=True, sharey=True)
axs = axs.flatten()

for i, rbm in enumerate(list_rbm_caracs):
    img = rbm.generer_image_RBM(gibbs_steps, 1)  # 1 image with 100 Gibbs steps
    
    img_reshaped = img[0].cpu().reshape(20, 16)
    
    axs[i].imshow(img_reshaped, cmap='Greys')
    axs[i].set_title(list(caracs.keys())[i])
    axs[i].axis('off')

# Adjust layout to prevent overlap
# plt.tight_layout()
plt.show()


## DBN

In [ ]:
liste_dbn_caracs = []
for carac in notebook.tqdm(list(caracs.keys()), desc="Characters", unit="char", leave=True):
            data = lire_alpha_digit([carac], path_data)     
            nb_pixels = data.shape[1]
            # les couches du dbn (de chaque rbm en soit)
            layers = [nb_pixels, neurons, neurons, neurons]

            dbn = DBN(layers=layers)
            dbn.train_DBN(x=data, epochs=[epochs], lr=learning_rate, plot=False, show_progress=True)
            liste_dbn_caracs.append(dbn)

In [ ]:
fig, axs = plt.subplots(4, 9, figsize=(10, 7), sharex=True, sharey=True)
axs = axs.flatten()

for i, dbn in enumerate(liste_dbn_caracs):
    img = dbn.generer_image_DBN(gibbs_steps, 1)  # 1 image with 100 Gibbs steps
    
    img_reshaped = img[0].cpu().reshape(20, 16)
    
    axs[i].imshow(img_reshaped, cmap='Greys')
    axs[i].set_title(list(caracs.keys())[i])
    axs[i].axis('off')

# Adjust layout to prevent overlap
# plt.tight_layout()
plt.show()

# Étude sur MNIST

## Chargement des données MNIST et processing

In [1]:
from principal_DNN_MNIST import *

In [2]:
import torchvision.datasets
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")


# if not exist, download mnist dataset
train_set = torchvision.datasets.MNIST(root='./data', train=True, transform=None, download=True)
test_set = torchvision.datasets.MNIST(root='./data', train=False, transform=None, download=True)
batch_size = 100

train_set.data = (train_set.data > 127).float()
test_set.data = (test_set.data > 127).float()


# Applatir les images (28*28) en vecteurs (784)
train_mnist = train_set.data.view(train_set.data.shape[0], -1).double().to(device)
test_mnist = test_set.data.view(test_set.data.shape[0], -1).double().to(device) 


In [3]:
# One Hot Encoding des labels
labels_train_mnist = torch.nn.functional.one_hot(train_set.targets).float().to(device)
labels_test_mnist = torch.nn.functional.one_hot(test_set.targets).float().to(device)


train_mnist.shape, labels_train_mnist.shape, test_mnist.shape, labels_test_mnist.shape


(torch.Size([60000, 784]),
 torch.Size([60000, 10]),
 torch.Size([10000, 784]),
 torch.Size([10000, 10]))

## Test du DNN sur MNIST

In [7]:

nb_pixels = train_mnist.shape[1]
neurons = 200
epochs = 200
learning_rate = 0.2
layers_dbn = [nb_pixels, neurons, neurons, neurons]
nb_classes = len(train_set.class_to_idx)
batch_size = 300

### DNN pre-entrainement

In [8]:
dnn_pretrain = DNN(layers_dbn, nb_classes=nb_classes)
dnn_pretrain.pretrain_DNN(x=train_mnist, epochs=[epochs], lr=learning_rate, show_progress=True)

Training DBN: 100%|██████████| 3/3 [09:18<00:00, 186.09s/RBM]


### Backpropagation sur 2 DNN

Non pré-entrainé

In [10]:
dnn_non_pretrain = DNN(layers_dbn, nb_classes=nb_classes)
dnn_non_pretrain.retropropagation(X=train_mnist, Y=labels_train_mnist, epochs=epochs, lr=learning_rate, batch_size=batch_size, show_progress=True)

Training DNN: 100%|██████████| 200/200 [05:14<00:00,  1.57s/epoch, Loss=0.182] 


Pre-entrainé 

In [9]:
dnn_pretrain.retropropagation(X=train_mnist, Y=labels_train_mnist, epochs=epochs, lr=learning_rate,batch_size=batch_size, show_progress=True)

Training DNN: 100%|██████████| 200/200 [05:12<00:00,  1.56s/epoch, Loss=0.0855]


### Test des deux DNN

In [11]:
dnn_pretrain.test_DNN(test_mnist, labels_test_mnist)

Taux d'erreur = 0.04280000180006027


(tensor(0.0428),
 tensor([[3.8927e-11, 5.0956e-11, 1.6646e-08,  ..., 1.0000e+00, 5.5229e-13,
          6.2407e-09],
         [1.0738e-09, 3.9581e-06, 1.0000e+00,  ..., 4.4147e-09, 5.1350e-08,
          3.6345e-14],
         [1.7822e-11, 9.9998e-01, 3.0964e-11,  ..., 2.2638e-05, 2.7290e-10,
          4.3201e-11],
         ...,
         [7.6290e-10, 3.6564e-10, 1.3343e-12,  ..., 6.8163e-08, 2.1116e-06,
          2.4690e-06],
         [1.2324e-06, 3.5321e-08, 4.3387e-10,  ..., 7.0744e-10, 2.0673e-03,
          7.0958e-09],
         [2.7588e-08, 7.9022e-08, 1.1453e-07,  ..., 6.8338e-11, 1.3528e-09,
          9.0773e-09]], dtype=torch.float64))

In [12]:
dnn_non_pretrain.test_DNN(test_mnist, labels_test_mnist)

Taux d'erreur = 0.05770000070333481


(tensor(0.0577),
 tensor([[6.0626e-08, 5.1642e-06, 1.4155e-03,  ..., 9.9843e-01, 1.7809e-07,
          1.0193e-05],
         [4.2668e-06, 2.1676e-04, 9.9880e-01,  ..., 5.8320e-05, 9.4321e-05,
          2.7142e-08],
         [5.0941e-09, 9.9997e-01, 4.1112e-07,  ..., 2.1902e-05, 6.9400e-07,
          1.2637e-07],
         ...,
         [5.2074e-05, 4.2364e-05, 2.0049e-03,  ..., 4.1218e-04, 2.0926e-04,
          4.6185e-04],
         [6.4549e-05, 4.8565e-03, 4.7160e-05,  ..., 6.6344e-06, 1.9525e-03,
          5.1822e-07],
         [9.4118e-05, 6.1847e-08, 3.9474e-06,  ..., 1.9364e-08, 5.8551e-06,
          5.1855e-08]], dtype=torch.float64))

debug

In [13]:
# from principal_DNN_MNIST import *
# import torchvision.datasets
# # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")


# # if not exist, download mnist dataset
# train_set = torchvision.datasets.MNIST(root='./data', train=True, transform=None, download=True)
# test_set = torchvision.datasets.MNIST(root='./data', train=False, transform=None, download=True)
# batch_size = 100

# train_set.data = (train_set.data > 127).float()
# test_set.data = (test_set.data > 127).float()


# # Applatir les images (28*28) en vecteurs (784)
# train_mnist = train_set.data.view(train_set.data.shape[0], -1).double().to(device)
# test_mnist = test_set.data.view(test_set.data.shape[0], -1).double().to(device) 

# # One Hot Encoding des labels
# labels_train_mnist = torch.nn.functional.one_hot(train_set.targets).float().to(device)
# labels_test_mnist = torch.nn.functional.one_hot(test_set.targets).float().to(device)


# train_mnist.shape, labels_train_mnist.shape, test_mnist.shape, labels_test_mnist.shape

# ## Test du DNN sur MNIST
# nb_pixels = train_mnist.shape[1]
# neurons = 200
# epochs = 200
# learning_rate = 0.1
# layers_dbn = [nb_pixels, neurons, neurons, neurons]
# nb_classes = len(train_set.class_to_idx)

# ### Backpropagation sur 2 DNN

# torch.unique(train_set.targets)
# dnn_non_pretrain = DNN(layers_dbn, nb_classes=nb_classes)
# dnn_non_pretrain.retropropagation(X=train_mnist, Y=labels_train_mnist, epochs=epochs, lr=learning_rate, show_progress=True)